In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# Шаг 1: Подготовка данных

# Функция для генерации последовательностей x и соответствующих y
def generate_sequences(length):
    x = np.random.randint(0, 10, size=length)  # Генерируем случайную последовательность x
    y = (x + x[0]) % 10  # Рассчитываем соответствующую последовательность y
    return x, y

# Генерация трех датасетов разной длины
dataset_lengths = [25, 75, 150]
datasets = []

for length in dataset_lengths:
    x_list, y_list = zip(*[generate_sequences(length) for _ in range(1000)])  # Создаем 1000 примеров для каждой длины
    x_array = np.array(x_list, dtype=np.float32)
    x_tensor = torch.tensor(x_array, dtype=torch.float32)
    y_tensor = torch.tensor(y_list, dtype=torch.float32)
    
    # Проверяем, что x и y имеют одинаковое количество элементов
    assert x_tensor.size(0) == y_tensor.size(0), "Size mismatch between x and y"
    
    dataset = TensorDataset(x_tensor, y_tensor)
    datasets.append(dataset)

# Шаг 2: Определение архитектур моделей

class MyRNN(nn.Module):
    def __init__(self):
        super(MyRNN, self).__init__()
        self.rnn = nn.RNN(input_size=1, hidden_size=64, batch_first=True)
        # Убираем слой fc
    def forward(self, x):
        out, _ = self.rnn(x)
        # Возвращаем выход RNN, который имеет размерность [batch_size, seq_len, hidden_size]
        return out


class MyLSTM(nn.Module):
    def __init__(self):
        super(MyLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=64, batch_first=True)
        self.fc = nn.Linear(64, 10)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

class MyGRU(nn.Module):
    def __init__(self):
        super(MyGRU, self).__init__()
        self.gru = nn.GRU(input_size=1, hidden_size=64, batch_first=True)
        self.fc = nn.Linear(64, 10)

    def forward(self, x):
        out, _ = self.gru(x)
        out = self.fc(out[:, -1, :])
        return out

# Шаг 3: Обучение моделей

# Функция для обучения модели
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        for x, y in train_loader:
            optimizer.zero_grad()

             # Изменение размерности x перед передачей в модель
            x = x.view(x.size(0), -1)

            outputs = model(x.unsqueeze(2))
            loss = criterion(outputs, y.long())
            loss.backward()
            optimizer.step()

# Шаг 4: Оценка производительности моделей

# Создаем заголовок для таблички с результатами
print(f"{'Модель':<10}{'Длина 25':<15}{'Длина 75':<15}{'Длина 150':<15}")

# Цикл по архитектурам моделей
for model_type, ModelClass in [("RNN", MyRNN), ("LSTM", MyLSTM), ("GRU", MyGRU)]:
    row = f"{model_type:<10}"
    
    # Цикл по датасетам
    for dataset in datasets:
        train_loader = DataLoader(dataset, batch_size=64, shuffle=True)
        model = ModelClass()
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        train_model(model, train_loader, criterion, optimizer)
        
        # Оценка производительности на тестовых данных
        test_loader = DataLoader(dataset, batch_size=64, shuffle=False)
        test_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for x, y in test_loader:
                outputs = model(x.unsqueeze(2))
                loss = criterion(outputs, y.long().long())
                test_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += y.size(0)
                correct += (predicted == y).sum().item()
        accuracy = 100 * correct / total
        row += f"{test_loss/len(test_loader):<15.4f}{accuracy:<15.2f}"
    print(row)


Модель    Длина 25       Длина 75       Длина 150      


RuntimeError: The size of tensor a (64) must match the size of tensor b (25) at non-singleton dimension 2

In [1]:
import torch
from torch import nn
import re
import random

# Загрузка и предобработка текстовых данных
text = open('nietzsche.txt', 'r', encoding='utf-8').read().lower()
text = re.sub(r'[^a-z ]', '', text)  # Удаление всех символов, кроме строчных букв и пробелов

# Создание списка уникальных символов и словарей для преобразования символов в индексы и наоборот
chars = sorted(list(set(text))) 
char_indices = {ch: i for i, ch in enumerate(chars)}
indices_char = {i: ch for i, ch in enumerate(chars)}

# Параметры для создания последовательных образцов и их целевых символов
maxlen = 20
step = 3

sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])  # Создание последовательных образцов длиной maxlen
    next_chars.append(text[i + maxlen])    # Запись целевого символа для каждого образца

# Создание тензоров для входных и целевых данных
X = torch.zeros((len(sentences), maxlen, len(chars)), dtype=torch.float).cuda()  # Перемещение на GPU
Y = torch.zeros((len(sentences)), dtype=torch.long).cuda()  # Перемещение на GPU

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1  # Применяем one-hot кодирование для символов
    Y[i] = char_indices[next_chars[i]]

# Определение модели LSTM и перемещение на GPU
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size)
        self.linear = nn.Linear(hidden_size, output_size)
        
    def forward(self, input, hidden):
        output, hidden = self.lstm(input, hidden)
        output = self.linear(output)
        return output, hidden

    def init_hidden(self, batch_size):
        return (torch.zeros(1, batch_size, self.hidden_size).cuda(),
                torch.zeros(1, batch_size, self.hidden_size).cuda())

# Инициализация модели и параметров тренировки
model = LSTM(len(chars), 128, len(chars)).cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Тренировка модели
for epoch in range(100):
    hidden = model.init_hidden(batch_size=1)  # Инициализация скрытого состояния
    optimizer.zero_grad()
    
    output, hidden = model(X.permute(1, 0, 2), hidden)  # Перестановка размерностей
    loss = criterion(output.permute(1, 0, 2).squeeze(), Y)  # Перестановка размерностей и использование .squeeze()
    loss.backward()
    
    optimizer.step()
    
    print('Эпоха: {}/{}. Потеря: {}'.format(epoch+1, 100, loss.item()))

# Генерация текста с использованием обученной модели
model.eval() 
generated = ''
sentence = text[random.randint(0, len(text)-maxlen-1):][:maxlen]
generated += sentence

for i in range(maxlen):
    x_inp = torch.tensor([char_indices[c] for c in sentence]).unsqueeze(0).cuda()
    hid = model.init_hidden(batch_size=1)
    output, hid = model(x_inp, hid)

    pred_index = output.argmax().item()
    predicted_char = indices_char[pred_index]

    generated += predicted_char
    sentence = sentence[1:] + predicted_char
    
print(generated)


RuntimeError: Expected hidden[0] size (1, 189802, 128), got [1, 1, 128]